In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    # rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [7]:
def model_loss(state_size, action_size, hidden_size,
               states, actions, next_states, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
#     dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
#                                                                     labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    # ##################################################### repeatable!
    # next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
    #                                reuse=True)
    # next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
    #                             reuse=True)
    # gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
    #                      action_size=action_size, reuse=True)
    # dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    # gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                 labels=tf.zeros_like(gQs3))) # 0-1
    # aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
    #                                                                  labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [8]:
def model_opt(a_loss, d_loss, a_loss2, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    #g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.dones, self.rates = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        d_loss=self.d_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [10]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [11]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [13]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
# gamma = 0.99                   # future reward discount

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, 
              d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [15]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                #rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                #rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                #if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                if len(dones) > 1:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            #             if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
            #                 print(np.count_nonzero(dones), len(dones), np.max(rates))
            #                 break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            #model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               #model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              #model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:1.4011 dloss:1.4941 aloss2:1.3162 exploreP:0.9980
Episode:1 meanR:21.5000 R:23.0000 rate:0.0460 aloss:1.3958 dloss:1.4578 aloss2:1.3414 exploreP:0.9958
Episode:2 meanR:19.0000 R:14.0000 rate:0.0280 aloss:1.4039 dloss:1.4673 aloss2:1.3428 exploreP:0.9944
Episode:3 meanR:20.0000 R:23.0000 rate:0.0460 aloss:1.4018 dloss:1.4638 aloss2:1.3302 exploreP:0.9921
Episode:4 meanR:19.0000 R:15.0000 rate:0.0300 aloss:1.3981 dloss:1.4315 aloss2:1.3623 exploreP:0.9906
Episode:5 meanR:21.8333 R:36.0000 rate:0.0720 aloss:1.3953 dloss:1.4120 aloss2:1.3636 exploreP:0.9871
Episode:6 meanR:20.7143 R:14.0000 rate:0.0280 aloss:1.3911 dloss:1.4078 aloss2:1.3672 exploreP:0.9857
Episode:7 meanR:19.8750 R:14.0000 rate:0.0280 aloss:1.3947 dloss:1.3809 aloss2:1.3832 exploreP:0.9844
Episode:8 meanR:19.5556 R:17.0000 rate:0.0340 aloss:1.3905 dloss:1.3821 aloss2:1.3910 exploreP:0.9827
Episode:9 meanR:27.7000 R:101.0000 rate:0.2020 aloss:1.3910 dloss:1.3543 aloss2:1.

Episode:80 meanR:23.4074 R:14.0000 rate:0.0280 aloss:1.4060 dloss:0.9385 aloss2:1.8334 exploreP:0.8290
Episode:81 meanR:23.6341 R:42.0000 rate:0.0840 aloss:1.4023 dloss:0.9178 aloss2:1.8612 exploreP:0.8256
Episode:82 meanR:23.5663 R:18.0000 rate:0.0360 aloss:1.4115 dloss:0.9086 aloss2:1.8764 exploreP:0.8241
Episode:83 meanR:23.4167 R:11.0000 rate:0.0220 aloss:1.3914 dloss:0.9101 aloss2:1.8986 exploreP:0.8232
Episode:84 meanR:23.2588 R:10.0000 rate:0.0200 aloss:1.4058 dloss:0.9051 aloss2:1.8976 exploreP:0.8224
Episode:85 meanR:23.1047 R:10.0000 rate:0.0200 aloss:1.3844 dloss:0.8814 aloss2:1.9065 exploreP:0.8216
Episode:86 meanR:23.1494 R:27.0000 rate:0.0540 aloss:1.4077 dloss:0.9056 aloss2:1.9091 exploreP:0.8194
Episode:87 meanR:23.0682 R:16.0000 rate:0.0320 aloss:1.4247 dloss:0.9029 aloss2:1.9171 exploreP:0.8181
Episode:88 meanR:23.0112 R:18.0000 rate:0.0360 aloss:1.4245 dloss:0.8849 aloss2:1.9314 exploreP:0.8167
Episode:89 meanR:22.8889 R:12.0000 rate:0.0240 aloss:1.4067 dloss:0.8992 

Episode:159 meanR:20.3800 R:12.0000 rate:0.0240 aloss:1.4873 dloss:0.6027 aloss2:3.4657 exploreP:0.7105
Episode:160 meanR:20.3100 R:10.0000 rate:0.0200 aloss:1.4739 dloss:0.6012 aloss2:3.4084 exploreP:0.7098
Episode:161 meanR:20.4400 R:24.0000 rate:0.0480 aloss:1.5006 dloss:0.5901 aloss2:3.5444 exploreP:0.7081
Episode:162 meanR:20.3200 R:14.0000 rate:0.0280 aloss:1.4527 dloss:0.6082 aloss2:3.4501 exploreP:0.7072
Episode:163 meanR:20.1000 R:11.0000 rate:0.0220 aloss:1.4232 dloss:0.6016 aloss2:3.6172 exploreP:0.7064
Episode:164 meanR:20.1300 R:16.0000 rate:0.0320 aloss:1.4483 dloss:0.5992 aloss2:3.5184 exploreP:0.7053
Episode:165 meanR:20.0800 R:11.0000 rate:0.0220 aloss:1.4978 dloss:0.5632 aloss2:3.5631 exploreP:0.7045
Episode:166 meanR:20.1100 R:14.0000 rate:0.0280 aloss:1.4841 dloss:0.5949 aloss2:3.5585 exploreP:0.7035
Episode:167 meanR:20.1500 R:27.0000 rate:0.0540 aloss:1.4388 dloss:0.5881 aloss2:3.6099 exploreP:0.7017
Episode:168 meanR:20.0800 R:10.0000 rate:0.0200 aloss:1.4527 dlo

Episode:238 meanR:17.1900 R:10.0000 rate:0.0200 aloss:1.4240 dloss:0.5450 aloss2:4.9762 exploreP:0.6232
Episode:239 meanR:17.1800 R:12.0000 rate:0.0240 aloss:1.4494 dloss:0.5243 aloss2:4.8840 exploreP:0.6225
Episode:240 meanR:17.1700 R:13.0000 rate:0.0260 aloss:1.4477 dloss:0.5015 aloss2:4.9043 exploreP:0.6217
Episode:241 meanR:17.1700 R:14.0000 rate:0.0280 aloss:1.4800 dloss:0.5316 aloss2:4.9816 exploreP:0.6208
Episode:242 meanR:17.1100 R:14.0000 rate:0.0280 aloss:1.4350 dloss:0.4813 aloss2:5.0352 exploreP:0.6200
Episode:243 meanR:17.1600 R:18.0000 rate:0.0360 aloss:1.4442 dloss:0.4722 aloss2:5.0096 exploreP:0.6189
Episode:244 meanR:16.8400 R:16.0000 rate:0.0320 aloss:1.4382 dloss:0.5019 aloss2:5.0117 exploreP:0.6179
Episode:245 meanR:16.7600 R:16.0000 rate:0.0320 aloss:1.4582 dloss:0.5233 aloss2:5.1416 exploreP:0.6169
Episode:246 meanR:16.7000 R:13.0000 rate:0.0260 aloss:1.4708 dloss:0.5225 aloss2:5.1225 exploreP:0.6161
Episode:247 meanR:16.5400 R:10.0000 rate:0.0200 aloss:1.4852 dlo

Episode:317 meanR:20.8700 R:14.0000 rate:0.0280 aloss:1.3878 dloss:0.4787 aloss2:5.3787 exploreP:0.5266
Episode:318 meanR:21.9000 R:112.0000 rate:0.2240 aloss:1.4074 dloss:0.5087 aloss2:5.3708 exploreP:0.5209
Episode:319 meanR:22.4600 R:71.0000 rate:0.1420 aloss:1.4233 dloss:0.5094 aloss2:5.4677 exploreP:0.5173
Episode:320 meanR:22.8100 R:49.0000 rate:0.0980 aloss:1.4012 dloss:0.5043 aloss2:5.5973 exploreP:0.5148
Episode:321 meanR:25.3200 R:266.0000 rate:0.5320 aloss:1.3936 dloss:0.5001 aloss2:5.5572 exploreP:0.5015
Episode:322 meanR:25.8900 R:107.0000 rate:0.2140 aloss:1.3907 dloss:0.5066 aloss2:5.5399 exploreP:0.4963
Episode:323 meanR:26.1900 R:51.0000 rate:0.1020 aloss:1.3864 dloss:0.5274 aloss2:5.5384 exploreP:0.4938
Episode:324 meanR:26.9700 R:94.0000 rate:0.1880 aloss:1.3853 dloss:0.4967 aloss2:5.5751 exploreP:0.4893
Episode:325 meanR:27.2800 R:42.0000 rate:0.0840 aloss:1.4042 dloss:0.4828 aloss2:5.5033 exploreP:0.4873
Episode:326 meanR:27.6200 R:44.0000 rate:0.0880 aloss:1.3759 

...

Episode:474 meanR:280.9100 R:208.0000 rate:0.4160 aloss:1.4093 dloss:0.8787 aloss2:3.5133 exploreP:0.0256
Episode:475 meanR:282.7300 R:215.0000 rate:0.4300 aloss:1.3941 dloss:0.9484 aloss2:3.5485 exploreP:0.0253
Episode:476 meanR:279.7400 R:201.0000 rate:0.4020 aloss:1.4243 dloss:0.8753 aloss2:3.5031 exploreP:0.0250
Episode:477 meanR:276.7900 R:205.0000 rate:0.4100 aloss:1.3587 dloss:1.0008 aloss2:3.5268 exploreP:0.0247
Episode:478 meanR:278.6400 R:364.0000 rate:0.7280 aloss:1.3873 dloss:0.9260 aloss2:3.4717 exploreP:0.0242
Episode:479 meanR:278.7400 R:275.0000 rate:0.5500 aloss:1.3774 dloss:0.9425 aloss2:3.4851 exploreP:0.0238
Episode:480 meanR:277.9800 R:241.0000 rate:0.4820 aloss:1.3674 dloss:1.0225 aloss2:3.4299 exploreP:0.0235
Episode:481 meanR:280.0400 R:416.0000 rate:0.8320 aloss:1.3755 dloss:0.9768 aloss2:3.3891 exploreP:0.0229
Episode:482 meanR:279.9000 R:229.0000 rate:0.4580 aloss:1.3693 dloss:0.9996 aloss2:3.3759 exploreP:0.0226
Episode:483 meanR:280.0600 R:266.0000 rate:0.5

Episode:552 meanR:236.6600 R:152.0000 rate:0.3040 aloss:1.3387 dloss:1.0390 aloss2:2.9272 exploreP:0.0125
Episode:553 meanR:235.7400 R:193.0000 rate:0.3860 aloss:1.3225 dloss:1.0877 aloss2:2.9489 exploreP:0.0124
Episode:554 meanR:236.2900 R:229.0000 rate:0.4580 aloss:1.3222 dloss:1.0918 aloss2:2.9446 exploreP:0.0124
Episode:555 meanR:233.1300 R:184.0000 rate:0.3680 aloss:1.3393 dloss:1.0371 aloss2:2.9533 exploreP:0.0123
Episode:556 meanR:233.3200 R:198.0000 rate:0.3960 aloss:1.2959 dloss:1.0963 aloss2:2.9361 exploreP:0.0123
Episode:557 meanR:232.4200 R:410.0000 rate:0.8200 aloss:1.3195 dloss:1.0494 aloss2:2.9580 exploreP:0.0122
Episode:558 meanR:233.1300 R:228.0000 rate:0.4560 aloss:1.3090 dloss:1.0956 aloss2:2.9651 exploreP:0.0121
Episode:559 meanR:233.6400 R:228.0000 rate:0.4560 aloss:1.3236 dloss:1.1182 aloss2:2.9362 exploreP:0.0121
Episode:560 meanR:234.3300 R:224.0000 rate:0.4480 aloss:1.3136 dloss:1.0850 aloss2:2.9380 exploreP:0.0120
Episode:561 meanR:233.9100 R:170.0000 rate:0.3

Episode:630 meanR:249.7000 R:214.0000 rate:0.4280 aloss:1.2066 dloss:1.2278 aloss2:2.4093 exploreP:0.0103
Episode:631 meanR:249.5500 R:193.0000 rate:0.3860 aloss:1.2175 dloss:1.2611 aloss2:2.3233 exploreP:0.0103
Episode:632 meanR:246.3000 R:175.0000 rate:0.3500 aloss:1.2281 dloss:1.2306 aloss2:2.3431 exploreP:0.0103
Episode:633 meanR:246.4000 R:181.0000 rate:0.3620 aloss:1.2123 dloss:1.2452 aloss2:2.3217 exploreP:0.0103
Episode:634 meanR:245.9700 R:159.0000 rate:0.3180 aloss:1.2265 dloss:1.2171 aloss2:2.3693 exploreP:0.0103
Episode:635 meanR:249.3700 R:500.0000 rate:1.0000 aloss:1.2075 dloss:1.2318 aloss2:2.3622 exploreP:0.0103
Episode:636 meanR:249.5900 R:231.0000 rate:0.4620 aloss:1.2269 dloss:1.2174 aloss2:2.3590 exploreP:0.0103
Episode:637 meanR:250.0800 R:218.0000 rate:0.4360 aloss:1.2075 dloss:1.2399 aloss2:2.3341 exploreP:0.0103
Episode:638 meanR:252.9900 R:500.0000 rate:1.0000 aloss:1.2065 dloss:1.2389 aloss2:2.3227 exploreP:0.0103
Episode:639 meanR:252.5900 R:196.0000 rate:0.3

Episode:708 meanR:313.6500 R:500.0000 rate:1.0000 aloss:1.1105 dloss:1.3466 aloss2:1.7661 exploreP:0.0100
Episode:709 meanR:314.3100 R:249.0000 rate:0.4980 aloss:1.1180 dloss:1.3575 aloss2:1.7366 exploreP:0.0100
Episode:710 meanR:317.4500 R:500.0000 rate:1.0000 aloss:1.1160 dloss:1.3509 aloss2:1.7411 exploreP:0.0100
Episode:711 meanR:320.5100 R:500.0000 rate:1.0000 aloss:1.1272 dloss:1.3503 aloss2:1.7414 exploreP:0.0100
Episode:712 meanR:318.8400 R:197.0000 rate:0.3940 aloss:1.1250 dloss:1.3565 aloss2:1.7269 exploreP:0.0100
Episode:713 meanR:321.4900 R:500.0000 rate:1.0000 aloss:1.1186 dloss:1.3504 aloss2:1.7261 exploreP:0.0100
Episode:714 meanR:321.4900 R:500.0000 rate:1.0000 aloss:1.1270 dloss:1.3549 aloss2:1.7141 exploreP:0.0100
Episode:715 meanR:323.7500 R:500.0000 rate:1.0000 aloss:1.1317 dloss:1.3496 aloss2:1.7156 exploreP:0.0100
Episode:716 meanR:325.8100 R:434.0000 rate:0.8680 aloss:1.1253 dloss:1.3520 aloss2:1.7016 exploreP:0.0100
Episode:717 meanR:328.6500 R:500.0000 rate:1.0

Episode:786 meanR:354.0600 R:500.0000 rate:1.0000 aloss:1.1053 dloss:1.2619 aloss2:1.8124 exploreP:0.0100
Episode:787 meanR:352.5500 R:349.0000 rate:0.6980 aloss:1.1091 dloss:1.2581 aloss2:1.8278 exploreP:0.0100
Episode:788 meanR:347.8400 R:29.0000 rate:0.0580 aloss:1.1202 dloss:1.2568 aloss2:1.8654 exploreP:0.0100
Episode:789 meanR:347.8400 R:500.0000 rate:1.0000 aloss:1.1073 dloss:1.2538 aloss2:1.8512 exploreP:0.0100
Episode:790 meanR:350.1200 R:500.0000 rate:1.0000 aloss:1.1066 dloss:1.2531 aloss2:1.8488 exploreP:0.0100
Episode:791 meanR:350.1200 R:500.0000 rate:1.0000 aloss:1.1064 dloss:1.2539 aloss2:1.8193 exploreP:0.0100
Episode:792 meanR:353.0100 R:500.0000 rate:1.0000 aloss:1.0969 dloss:1.2402 aloss2:1.8611 exploreP:0.0100
Episode:793 meanR:353.0100 R:500.0000 rate:1.0000 aloss:1.1054 dloss:1.2365 aloss2:1.8369 exploreP:0.0100
Episode:794 meanR:348.7600 R:75.0000 rate:0.1500 aloss:1.1051 dloss:1.2444 aloss2:1.8748 exploreP:0.0100
Episode:795 meanR:351.4500 R:500.0000 rate:1.000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
# eps, arr = np.array(dloss_list).T
# smoothed_arr = running_mean(arr, 10)
# plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
# plt.plot(eps, arr, color='grey', alpha=0.3)
# plt.xlabel('Episode')
# plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.